# Trabalho de Conformidade do Curso de Dados Abertos para Controle Social

O objetivo do presente trabalho é verificar quais empresas de pessoas politicamente expostas que receberam recursos dos próprios municípios em que têm mandatos.

Para isso serão utilizado dados do Portal de Transparência Municipal do TCESP, base de dados públicos de CNPJ da SRF e dados do Portal da Transparência da CGU.

**Atenção:** Como o presente trabalho é para fins educacionais, os nomes, CPFs e CNPJs foram alterados, mas as estruturas são idênticas às das tabelas originais.

## Importar as bibliotecas utilizadas

In [49]:
import os
import shutil

import folium
import pandas as pd
import sqlalchemy

## Carregar Dados

In [2]:
diretorio_dados = 'dados'

### Carregar dados de banco

O banco de dados possui três tabelas: despesas_consolidado, empresas e sócios

In [3]:
arquivo_sqlite = 'banco_trabalho06.db'
filepath_arquivo_sqlite = os.path.join(diretorio_dados, arquivo_sqlite)

In [4]:
url_banco = "sqlite:///{}".format(filepath_arquivo_sqlite)
engine = sqlalchemy.create_engine(url_banco)

#### Despesas 
É a totalização de valores pagos a empresas por município, valores aproximadamente acima de 100 mil reais.

A consolidação foi realizada a partir da tabela despesas do TCE/SP. Os nomes de empresas, cnpjs e municípios foram alterados.

A coluna **ds_municipio** é o nome do município fictício, **nr_identificador_despesa** o CNPJ da empresa que recebeu o valor da coluna soma e **ds_despesa** a razão social fictícia da empresa.

In [5]:
sql = 'SELECT * FROM despesas_consolidado'
despesas = pd.read_sql(sql, engine, index_col=None)

Aqui podemos visualizar uma amostra dos dados

In [6]:
despesas.sample(5)

,ds_municipio,nr_identificador_despesa,ds_despesa,soma
3100,AMANTE,00466957000174,LAIA SISTEMAS,1303111.87
3993,CHIZZOTTI,08966092000208,EDGAR GAS LTDA,1132683.60
17551,NEJELSKI,37143239000115,HADRIEN PARTICIPACOES,173656.40
24485,LIENSTADT,65036061000103,IMRE CONTABILIDADE,98721.81
6754,COSTIM,03182500000100,ALLISON MODAS,552861.74


Com o comando abaixo, podemos ver que a tabela possui 25.885 registros; não possui dados nulos e há apenas uma coluna com dados numéricos.

In [7]:
despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25885 entries, 0 to 25884
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ds_municipio              25885 non-null  object 
 1   nr_identificador_despesa  25885 non-null  object 
 2   ds_despesa                25885 non-null  object 
 3   soma                      25885 non-null  float64
dtypes: float64(1), object(3)
memory usage: 809.0+ KB


#### Empresas 
Tabela de empresas da receita federal.

Possui dados alterados, mas a estrutura das colunas é semelhante a base pública da Receita Federal.

In [8]:
sql = 'SELECT * FROM empresas'
empresas = pd.read_sql(sql, engine, index_col=None)

Aqui podemos visualizar uma amostra dos dados

In [9]:
empresas.sample(5)

,cnpj,matriz_filial,razao_social,nome_fantasia,situacao,data_situacao,motivo_situacao,nm_cidade_exterior,cod_pais,nome_pais,...,email,qualif_resp,capital_social,porte,opc_simples,data_opc_simples,data_exc_simples,opc_mei,sit_especial,data_sit_especial
1395,53490452000338,2,JADE ACESSORIOS LTDA,,02,20200101,00,,,,...,,,0.000000e+00,05,0,,,,,
3649,02522238000106,1,CAMILLA PARTICIPACOES,,02,20200101,37,,,,...,,,0.000000e+00,05,0,,,N,,
1509,04697232000148,1,JIMENA PARTICIPACOES LTDA,,02,20200101,01,,,,...,,,0.000000e+00,01,0,,,N,,
4587,05060838000169,1,RAPHAEL AGROPECUARIA,,02,20200101,00,,,,...,,,1.191487e+07,05,0,,,N,,
5363,05259724000177,1,CARLOS SAUDE EPP,,02,20200101,00,,,,...,,,0.000000e+00,05,0,,,N,,


Com o comando abaixo, podemos ver que a tabela possui 5.779 registros; não possui dados nulos e há apenas uma coluna com dados numéricos.

In [10]:
empresas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cnpj                5779 non-null   object 
 1   matriz_filial       5779 non-null   object 
 2   razao_social        5779 non-null   object 
 3   nome_fantasia       5779 non-null   object 
 4   situacao            5779 non-null   object 
 5   data_situacao       5779 non-null   object 
 6   motivo_situacao     5779 non-null   object 
 7   nm_cidade_exterior  5779 non-null   object 
 8   cod_pais            5779 non-null   object 
 9   nome_pais           5779 non-null   object 
 10  cod_nat_juridica    5779 non-null   object 
 11  data_inicio_ativ    5779 non-null   object 
 12  cnae_fiscal         5779 non-null   object 
 13  tipo_logradouro     5779 non-null   object 
 14  logradouro          5779 non-null   object 
 15  numero              5779 non-null   object 
 16  comple

#### Sócios
Tabela de sócios da receita federal.

Possui dados alterados, mas a estrutura das colunas é semelhante a base pública da Receita Federal.

In [11]:
sql = 'SELECT * FROM socios'
socios = pd.read_sql(sql, engine, index_col=None)

Aqui podemos visualizar uma amostra dos dados

In [12]:
socios.sample(5)

,cnpj,tipo_socio,nome_socio,cnpj_cpf_socio,cod_qualificacao,perc_capital,data_entrada,cod_pais_ext,nome_pais_ext,cpf_repres,nome_repres,cod_qualif_repres
1711,52429387000171,2,EDISON FREJDA FRALANJE,***072018**,49,0.0,20200101,,,,,00
803,74233517000151,2,SPERETA RHANDS RUVAMAN,***564808**,22,0.0,20200101,,,,,00
5774,15448396000131,2,WILLAN EVANDRO ALBERTINSKY RAGO,***202288**,49,0.0,20200101,,,,,00
591,55610968000123,2,SPERETA UNAIDE ESTAVAS,***184928**,16,0.0,20200101,,,,,00
2494,02145121000568,2,HELIO RUSHANSKY CORAT,***331538**,59,0.0,20200101,,,,,00


Com o comando abaixo, podemos ver que a tabela possui 5.825 registros; não possui dados nulos e há apenas uma coluna com dados numéricos.

In [13]:
socios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5825 entries, 0 to 5824
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cnpj               5825 non-null   object 
 1   tipo_socio         5825 non-null   object 
 2   nome_socio         5825 non-null   object 
 3   cnpj_cpf_socio     5825 non-null   object 
 4   cod_qualificacao   5825 non-null   object 
 5   perc_capital       5825 non-null   float64
 6   data_entrada       5825 non-null   object 
 7   cod_pais_ext       5825 non-null   object 
 8   nome_pais_ext      5825 non-null   object 
 9   cpf_repres         5825 non-null   object 
 10  nome_repres        5825 non-null   object 
 11  cod_qualif_repres  5825 non-null   object 
dtypes: float64(1), object(11)
memory usage: 546.2+ KB


### Carregar dados de Prefeitos e Vereadores de São Paulo

In [14]:
arquivo_peps = 'PEP_SP.csv'
filepath_arquivo_pep = os.path.join(diretorio_dados, arquivo_peps)

In [15]:
peps = pd.read_csv(filepath_arquivo_pep, sep=';')

Aqui podemos visualizar uma amostra dos dados

In [16]:
peps.sample(5)

,CPF,Nome_PEP,Sigla_Função,Descrição_Função,Nível_Função,Nome_Órgão,Data_Início_Exercício,Data_Fim_Exercício,Data_Fim_Carência,uf_orgao
3791,***.483.568-**,PIRASSOLLI PHREDERICO HAUACH COLPAERT,VEREAD,VEREADOR,NaN,MUN. DE IALONGO-SP,01/01/2017,31/12/2020,31/12/2025,SP
5921,***.764.618-**,MARCO SPERETA DE DINHEIRO,VEREAD,VEREADOR,NaN,MUN. DE CALMANOVICI-SP,01/01/2017,31/12/2020,31/12/2025,SP
6337,***.819.168-**,ROMILDO BENEDITO CESTONE,VEREAD,VEREADOR,NaN,MUN. DE MINARELLI-SP,01/01/2017,31/12/2020,31/12/2025,SP
1133,***.140.633-**,SPERETA EISSAKU SUSAKO NIRISMAR,VEREAD,VEREADOR,NaN,MUN. DE LEUTPRECHT-SP,01/01/2017,31/12/2020,31/12/2025,SP
2022,***.248.328-**,CATARINA ZIELAK COMNENE,VEREAD,VEREADOR,NaN,MUN. DE TOMPOROWSKI-SP,01/01/2017,31/12/2020,31/12/2025,SP


Com o comando abaixo, podemos ver que a tabela possui 7.690 registros; possui apenas o coluna **Nível_Função** com dados nulos.

In [17]:
peps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7690 entries, 0 to 7689
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CPF                    7690 non-null   object 
 1   Nome_PEP               7690 non-null   object 
 2   Sigla_Função           7690 non-null   object 
 3   Descrição_Função       7690 non-null   object 
 4   Nível_Função           0 non-null      float64
 5   Nome_Órgão             7690 non-null   object 
 6   Data_Início_Exercício  7690 non-null   object 
 7   Data_Fim_Exercício     7690 non-null   object 
 8   Data_Fim_Carência      7690 non-null   object 
 9   uf_orgao               7690 non-null   object 
dtypes: float64(1), object(9)
memory usage: 600.9+ KB


## Execução

In [18]:
# Definir diretorio saida
diretorio_saida_dados = 'saida'

# Remover diretorio saida, se houver
shutil.rmtree(diretorio_saida_dados, ignore_errors=True)

# Criar diretorio saida
os.makedirs(diretorio_saida_dados, exist_ok=True)

### Tratamento dos dados

Alguns dados precisam ser tratados antes da análise.

#### Empresa

A coluna **data_situacao** está no formato YYYMMDD, para facilitar a análise, vamos criar a coluna **data_situacao_normalizado** com o formato YYYY-MM-DD

In [19]:
empresas['data_situacao'].sample(5)

377     20200101
4145    20200101
2933    20200101
360     20200101
2079    20200101
Name: data_situacao, dtype: object

In [20]:
empresas['data_situacao_normalizado'] = pd.to_datetime(empresas['data_situacao']).dt.date

In [21]:
empresas['data_situacao_normalizado'].sample(5)

248     2020-01-01
809     2020-01-01
5121    2020-01-01
4971    2020-01-01
4371    2020-01-01
Name: data_situacao_normalizado, dtype: object

#### Socios

A coluna **data_entrada** está no formato YYYMMDD, para facilitar a análise, vamos criar a coluna **data_entrada_normalizado** com o formato YYYY-MM-DD

In [22]:
socios['data_entrada'].sample(5)

4003    20200101
4709    20200101
1376    20200101
1329    20200101
249     20200101
Name: data_entrada, dtype: object

In [23]:
socios['data_entrada_normalizado'] = pd.to_datetime(socios['data_entrada']).dt.date

In [24]:
socios['data_entrada_normalizado'].sample(5)

4315    2020-01-01
3430    2020-01-01
3989    2020-01-01
4875    2020-01-01
146     2020-01-01
Name: data_entrada_normalizado, dtype: object

### Pessoas Politicamente Expostas

A coluna **CPF** possuem pontuação, vamos criar a coluna **CPF_normalizado** sem elas.

In [25]:
peps['CPF'].sample(5)

6518    ***.846.308-**
1566    ***.190.168-**
3191    ***.394.678-**
7519    ***.977.968-**
2827    ***.344.098-**
Name: CPF, dtype: object

In [26]:
peps['CPF_normalizado'] = peps['CPF'].str.replace(".", "", regex=False).str.replace("-", "", regex=False)

In [27]:
peps['CPF_normalizado'].sample(5)

5949    ***767478**
4126    ***527148**
5479    ***708538**
1253    ***154988**
2219    ***273648**
Name: CPF_normalizado, dtype: object

Na coluna **Nome_Órgão** vamos filtrar dados extras e manter apenas o nome do município na coluna **Municipio_normalizado**

In [28]:
peps['Nome_Órgão'].sample(5)

208     MUN. DE CRESCENTE-SP
4245       MUN. DE DALAGO-SP
333      MUN. DE HIROHARA-SP
3783      MUN. DE COMETTI-SP
5917      MUN. DE NEDEHER-SP
Name: Nome_Órgão, dtype: object

In [29]:
peps['Municipio_normalizado'] = peps['Nome_Órgão'].str.replace("MUN. DE ", "", regex=False).str.replace("-SP", "", regex=False)

In [30]:
peps['Municipio_normalizado'].sample(5)

273          MOO
1569        ALLO
7166    RICOMINI
7226    BETIOLLO
759        MAFIA
Name: Municipio_normalizado, dtype: object

As coluna **Data_Início_Exercício**, **Data_Fim_Exercício** e **Data_Fim_Carência** estão no formato YYYY/MM/DD, para facilitar a análise, vamos criar a colunas **Data_Inicio_Exercicio_normalizado**, **Data_Fim_Exercicio_normalizado** e **Data_Fim_Carência_normalizado** com o formato YYYY-MM-DD

In [31]:
peps['Data_Início_Exercício'].sample(5)

1397    01/01/2017
5624    01/01/2017
176     01/01/2017
582     01/01/2017
6727    01/01/2017
Name: Data_Início_Exercício, dtype: object

In [32]:
peps['Data_Fim_Exercício'].sample(5)

1324    31/12/2020
5281    31/12/2020
5967    31/12/2020
5474    31/12/2020
5321    31/12/2020
Name: Data_Fim_Exercício, dtype: object

In [33]:
peps['Data_Fim_Carência'].sample(5)

877     31/12/2025
263     31/12/2025
6513    31/12/2025
974     31/12/2025
1064    31/12/2025
Name: Data_Fim_Carência, dtype: object

In [34]:
peps['Data_Inicio_Exercicio_normalizado'] = pd.to_datetime(peps['Data_Início_Exercício']).dt.date
peps['Data_Fim_Exercicio_normalizado'] = pd.to_datetime(peps['Data_Fim_Exercício']).dt.date
peps['Data_Fim_Carência_normalizado'] = pd.to_datetime(peps['Data_Fim_Carência']).dt.date

In [35]:
peps['Data_Inicio_Exercicio_normalizado'].sample(5)

1540    2017-01-01
7094    2017-01-01
5596    2017-01-01
2464    2017-01-01
6526    2017-01-01
Name: Data_Inicio_Exercicio_normalizado, dtype: object

In [36]:
peps['Data_Fim_Exercicio_normalizado'].sample(5)

3234    2020-12-31
6381    2020-12-31
4109    2020-12-31
1565    2020-12-31
3414    2020-12-31
Name: Data_Fim_Exercicio_normalizado, dtype: object

In [37]:
peps['Data_Fim_Carência_normalizado'].sample(5)

3632    2025-12-31
6474    2025-12-31
2369    2025-12-31
6117    2025-12-31
5005    2025-12-31
Name: Data_Fim_Carência_normalizado, dtype: object

### Gerar Banco de Dados

#### Copiar banco original para diretorio de saida

In [38]:
arquivo_banco_saida = 'output.db'
filepath_arquivo_banco_saida = os.path.join(diretorio_saida_dados, arquivo_banco_saida)
shutil.copyfile(filepath_arquivo_sqlite, filepath_arquivo_banco_saida)

'saida\\output.db'

#### Gravar tabela de prefeitos_vereadores em banco gerado

In [39]:
url_banco = "sqlite:///{}".format(filepath_arquivo_banco_saida)
engine_banco_saida = sqlalchemy.create_engine(url_banco)

In [40]:
peps.to_sql('peps', con=engine_banco_saida)

### Consultar Empresas Relacionadas com Pessoas Politicamente Expostas que Receberam Recursos

In [41]:
sql = 'SELECT \
  des.ds_municipio AS [Nome_Municipio_Ficticio], \
  des.nr_identificador_despesa AS [CNPJ_Empresa_Beneficiaria], \
  des.ds_despesa AS [Razao_Social_Empresa_Beneficiaria], \
  des.soma AS [Valor_Total], \
  emp.cnpj AS [CNPJ_RFB], \
  emp.razao_social AS [Razao_Social_RFB], \
  CASE emp.situacao \
    WHEN "01" THEN "NULA" \
    WHEN "02" THEN "ATIVA" \
    WHEN "03" THEN "SUSPENSA" \
    WHEN "04" THEN "INAPTA" \
    WHEN "08" THEN "BAIXADA" \
  END AS [Situacao_Cadastral_RFB], \
  CAST(emp.data_situacao AS DATE) AS [Data_Situacao_Cadastral_RFB], \
  emp.cod_nat_juridica AS [Codigo_Natureza_Juridica_RFB], \
  CAST(emp.data_inicio_ativ AS DATE) AS [Data_Inicio_Atividade_RFB], \
  emp.cnae_fiscal AS [CNAE_Fiscal_RFB], \
  emp.municipio AS [Municipio_RFB], \
  emp.uf AS [UF_RFB], \
  soc.nome_socio AS [Nome_Socio_RFB], \
  soc.cnpj_cpf_socio AS [CPF_Socio_RFB], \
  CAST(soc.data_entrada AS DATE) AS [Data_Entrada_Socio_RFB], \
  pep.Nome_PEP AS [Nome_PEP], \
  pep.CPF_normalizado AS [CPF_PEP], \
  pep.Descrição_Função AS [Funcao_PEP], \
  pep.Data_Inicio_Exercicio_normalizado AS [Data_Inicio_PEP], \
  pep.Data_Fim_Exercicio_normalizado AS [Data_Fim_PEP], \
  pep.Data_Fim_Carência_normalizado AS [Data_Carencia_PEP], \
  pep.Municipio_normalizado AS [Municipio_PEP], \
  pep.uf_orgao AS [UF_PEP] \
FROM despesas_consolidado des \
JOIN empresas emp ON emp.cnpj = des.nr_identificador_despesa \
JOIN socios soc ON soc.cnpj = emp.cnpj \
JOIN peps pep ON pep.CPF_normalizado = soc.cnpj_cpf_socio AND pep.Nome_PEP = soc.nome_socio'

In [42]:
empresas_peps = pd.read_sql(sql, engine_banco_saida, index_col=None)

In [43]:
empresas_peps.sample(5)

,Nome_Municipio_Ficticio,CNPJ_Empresa_Beneficiaria,Razao_Social_Empresa_Beneficiaria,Valor_Total,CNPJ_RFB,Razao_Social_RFB,Situacao_Cadastral_RFB,Data_Situacao_Cadastral_RFB,Codigo_Natureza_Juridica_RFB,Data_Inicio_Atividade_RFB,...,CPF_Socio_RFB,Data_Entrada_Socio_RFB,Nome_PEP,CPF_PEP,Funcao_PEP,Data_Inicio_PEP,Data_Fim_PEP,Data_Carencia_PEP,Municipio_PEP,UF_PEP
8,VALLIS,55815300000213,AMINE IMOBILIARIOS,747080.25,55815300000213,AMINE IMOBILIARIOS,ATIVA,20200101,3999,19870728,...,***890658**,20200101,AMARILDO SABATE KAILER,***890658**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,SOLIZ,SP
44,NIELAND,06273922000124,MOMOKA ARTISTICAS LTDA,195474.65,06273922000124,MOMOKA ARTISTICAS LTDA,ATIVA,20200101,3999,20000214,...,***923478**,20200101,GEOVANISA TERSINHA BERTASSONI,***923478**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,SACCOL,SP
43,DALO,10240793000137,JOAO LUCAS ELETRONICOS SA,221294.85,10240793000137,JOAO LUCAS ELETRONICOS SA,ATIVA,20200101,3999,20130111,...,***502138**,20200101,CAUME UNAIDE ABASILIA PHREDERICO ZIELAK,***502138**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,BREIA,SP
22,VITT,06273922000124,MOMOKA ARTISTICAS LTDA,421712.59,06273922000124,MOMOKA ARTISTICAS LTDA,ATIVA,20200101,3999,20000214,...,***923478**,20200101,GEOVANISA TERSINHA BERTASSONI,***923478**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,SACCOL,SP
28,SITA,49524688000199,IBAI VESTUARIO,339078.90,49524688000199,IBAI VESTUARIO,ATIVA,20200101,2038,20201019,...,***460278**,20200101,SAVIOLO LUYTEN SMERIERI,***460278**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,SITA,SP


#### Salvar dados

In [44]:
arquivo_saida = 'empresas_peps.csv'
filepath = os.path.join(diretorio_saida_dados, arquivo_saida)
empresas_peps.to_csv(filepath, index=False)

### Consultar Empresas Relacionadas com Pessoas Politicamente Expostas que Receberam Recursos dos Municípios que Possuem Mandatos

In [45]:
sql = 'SELECT \
  des.ds_municipio AS [Nome_Municipio_Ficticio], \
  des.nr_identificador_despesa AS [CNPJ_Empresa_Beneficiaria], \
  des.ds_despesa AS [Razao_Social_Empresa_Beneficiaria], \
  des.soma AS [Valor_Total], \
  emp.cnpj AS [CNPJ_RFB], \
  emp.razao_social AS [Razao_Social_RFB], \
  CASE emp.situacao \
    WHEN "01" THEN "NULA" \
    WHEN "02" THEN "ATIVA" \
    WHEN "03" THEN "SUSPENSA" \
    WHEN "04" THEN "INAPTA" \
    WHEN "08" THEN "BAIXADA" \
  END AS [Situacao_Cadastral_RFB], \
  CAST(emp.data_situacao AS DATE) AS [Data_Situacao_Cadastral_RFB], \
  emp.cod_nat_juridica AS [Codigo_Natureza_Juridica_RFB], \
  CAST(emp.data_inicio_ativ AS DATE) AS [Data_Inicio_Atividade_RFB], \
  emp.cnae_fiscal AS [CNAE_Fiscal_RFB], \
  emp.municipio AS [Municipio_RFB], \
  emp.uf AS [UF_RFB], \
  soc.nome_socio AS [Nome_Socio_RFB], \
  soc.cnpj_cpf_socio AS [CPF_Socio_RFB], \
  CAST(soc.data_entrada AS DATE) AS [Data_Entrada_Socio_RFB], \
  pep.Nome_PEP AS [Nome_PEP], \
  pep.CPF_normalizado AS [CPF_PEP], \
  pep.Descrição_Função AS [Funcao_PEP], \
  pep.Data_Inicio_Exercicio_normalizado AS [Data_Inicio_PEP], \
  pep.Data_Fim_Exercicio_normalizado AS [Data_Fim_PEP], \
  pep.Data_Fim_Carência_normalizado AS [Data_Carencia_PEP], \
  pep.Municipio_normalizado AS [Municipio_PEP], \
  pep.uf_orgao AS [UF_PEP] \
FROM despesas_consolidado des \
JOIN empresas emp ON emp.cnpj = des.nr_identificador_despesa \
JOIN socios soc ON soc.cnpj = emp.cnpj \
JOIN peps pep ON pep.CPF_normalizado = soc.cnpj_cpf_socio AND pep.Nome_PEP = soc.nome_socio \
WHERE emp.uf = pep.uf_orgao AND emp.municipio = pep.Municipio_normalizado'

In [46]:
empresas_peps_mesmo_municipio = pd.read_sql(sql, engine_banco_saida, index_col=None)

In [47]:
empresas_peps_mesmo_municipio.sample(5)

,Nome_Municipio_Ficticio,CNPJ_Empresa_Beneficiaria,Razao_Social_Empresa_Beneficiaria,Valor_Total,CNPJ_RFB,Razao_Social_RFB,Situacao_Cadastral_RFB,Data_Situacao_Cadastral_RFB,Codigo_Natureza_Juridica_RFB,Data_Inicio_Atividade_RFB,...,CPF_Socio_RFB,Data_Entrada_Socio_RFB,Nome_PEP,CPF_PEP,Funcao_PEP,Data_Inicio_PEP,Data_Fim_PEP,Data_Carencia_PEP,Municipio_PEP,UF_PEP
5,PATRUCCO,12480500000115,VICENTE MODAS,306932.06,12480500000115,VICENTE MODAS,BAIXADA,20200101,2305,20140218,...,***178358**,20200101,JOAO YPEI DE KAARA,***178358**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,GAZETA,SP
2,SUCHOBKOW,56776669000107,FINN TECNOLOGIA,559375.94,56776669000107,FINN TECNOLOGIA,ATIVA,20200101,3999,19860918,...,***078588**,20200101,IDEVAL DELU STRAPPA,***078588**,VEREADOR,2017-01-01,2020-12-31,2025-12-31,SUCHOBKOW,SP
3,HACKEL,04578574000175,SOMA TURISMO EPP,513335.60,04578574000175,SOMA TURISMO EPP,ATIVA,20200101,2062,19960117,...,***149208**,20200101,CLOZOE SPERETA DANIEL,***149208**,PREFEITO,2017-01-01,2020-12-31,2025-12-31,HACKEL,SP
0,KANDELMAN,51653764000156,ROBERTS INFORMACAO EPP,14121381.99,51653764000156,ROBERTS INFORMACAO EPP,ATIVA,20200101,3999,19661022,...,***708958**,20200101,ADEMIR FABRISIA,***708958**,VEREADOR,2017-01-01,2020-12-31,2025-12-31,KANDELMAN,SP
6,ALVERNE,47817109002513,CLAIRE SUPERMERCADO,311980.65,47817109002513,CLAIRE SUPERMERCADO,ATIVA,20200101,2062,20050723,...,***199388**,20200101,UNAIDE ALBERTO DAMAR,***199388**,VEREADOR,2017-01-01,2020-12-31,2025-12-31,ALVERNE,SP


#### Salvar dados

In [48]:
arquivo_saida = 'empresas_peps_mesmo_municipio.csv'
filepath = os.path.join(diretorio_saida_dados, arquivo_saida)
empresas_peps_mesmo_municipio.to_csv(filepath, index=False)

In [51]:
arquivo_municipios = 'Municipios_SP_paridade.csv'
filepath_arquivo_municipios = os.path.join(diretorio_dados, arquivo_municipios)

In [52]:
municipios = pd.read_csv(filepath_arquivo_municipios, sep='|')

In [54]:
empresas_peps_coord = pd.merge(empresas_peps, municipios, how="inner", left_on='Nome_Municipio_Ficticio', right_on='Municipio_ficticio')

In [55]:
empresas_peps_coord.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 72
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Nome_Municipio_Ficticio            73 non-null     object 
 1   CNPJ_Empresa_Beneficiaria          73 non-null     object 
 2   Razao_Social_Empresa_Beneficiaria  73 non-null     object 
 3   Valor_Total                        73 non-null     float64
 4   CNPJ_RFB                           73 non-null     object 
 5   Razao_Social_RFB                   73 non-null     object 
 6   Situacao_Cadastral_RFB             73 non-null     object 
 7   Data_Situacao_Cadastral_RFB        73 non-null     int64  
 8   Codigo_Natureza_Juridica_RFB       73 non-null     object 
 9   Data_Inicio_Atividade_RFB          73 non-null     int64  
 10  CNAE_Fiscal_RFB                    73 non-null     object 
 11  Municipio_RFB                      73 non-null     object 
 

In [56]:
empresas_peps_coord.sample(5)

,Nome_Municipio_Ficticio,CNPJ_Empresa_Beneficiaria,Razao_Social_Empresa_Beneficiaria,Valor_Total,CNPJ_RFB,Razao_Social_RFB,Situacao_Cadastral_RFB,Data_Situacao_Cadastral_RFB,Codigo_Natureza_Juridica_RFB,Data_Inicio_Atividade_RFB,...,Municipio_PEP,UF_PEP,ID,IBGE_7,IBGE_6,Municipio_real,UF,Latitude,Longitude,Municipio_ficticio
12,CAVICHIOLI,55815300000213,AMINE IMOBILIARIOS,568608.30,55815300000213,AMINE IMOBILIARIOS,ATIVA,20200101,3999,19870728,...,SOLIZ,SP,132,3511607,351160,CESARIO LANGE,SP,-23.227,-47.953,CAVICHIOLI
32,DALAGO,01379160000172,ALEKSANDRA INDUSTRIAS,285251.53,01379160000172,ALEKSANDRA INDUSTRIAS,ATIVA,20200101,2062,20070307,...,SAO PAULO,SP,181,3515707,351570,FERRAZ DE VASCONCELOS,SP,-23.541,-46.369,DALAGO
9,COSLOVSKY,55815300000213,AMINE IMOBILIARIOS,747635.06,55815300000213,AMINE IMOBILIARIOS,ATIVA,20200101,3999,19870728,...,SOLIZ,SP,171,3515152,351515,ENGENHEIRO COELHO,SP,-22.488,-47.215,COSLOVSKY
23,KASTROPIL,69669464000188,MARTINA ENGENHARIA EPP,423179.55,69669464000188,MARTINA ENGENHARIA EPP,ATIVA,20200101,2062,19910518,...,UJIHARA,SP,308,3527256,352725,LOURDES,SP,-20.967,-50.224,KASTROPIL
25,HORST,06380418000123,JOSHUA SEGUROS,418912.16,06380418000123,JOSHUA SEGUROS,ATIVA,20200101,2062,20000423,...,UJIHARA,SP,281,3524709,352470,JAGUARIUNA,SP,-22.706,-46.986,HORST


In [61]:
mapa_plot = folium.Map(location=[empresas_peps_coord.Latitude.mean(), empresas_peps_coord.Longitude.mean()], zoom_start=7, control_scale=True)
for index, location_info in empresas_peps_coord.iterrows():
    description = "A empresa " + location_info["Razao_Social_Empresa_Beneficiaria"] + " (CNPJ " + location_info["CNPJ_Empresa_Beneficiaria"] + "), relacionada ao PEP " + location_info["Nome_PEP"] + ", recebeu R$ " + str(location_info["Valor_Total"])
    description.splitlines(True)
    folium.Marker(
        [location_info["Latitude"], location_info["Longitude"]],
        popup=description,
        tooltip=description,
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(mapa_plot)
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Mapa (fictício) de empresas relacionadas com PEPs</b></h3>
             '''
mapa_plot.get_root().html.add_child(folium.Element(title_html))
mapa_plot